# mount google drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# set working directory

In [114]:
import os
os.chdir("/home/michael/github/CC0325/codes/matmat/")

In [115]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sympy as sp
import seaborn as sns
# set theme, white background
sns.set_theme(style="whitegrid")

In [116]:
def plot_polynomial_fit(df, column_name, column_values, degree=2):
    # Fit a polynomial to the data
    coefficients = np.polyfit(df[column_name], df[column_values], degree)
    poly = np.poly1d(coefficients)

    # Show the polynomial fit equation
    x = sp.Symbol('x')
    poly_equation = sp.Poly(poly(x), x)
    print(f"Polynomial Fit Equation: {poly_equation}")

    # Plot the data points
    sns.scatterplot(data=df, x=column_name, y=column_values, marker='o', label='Data Points')

    # Plot the polynomial fit, dashed line
    sns.lineplot(data=df, x=column_name, y=poly(df[column_name]), label='Poly Fit', linestyle='--')

    plt.legend()
    plt.show()


In [ ]:
!make clean
!make run

# read the output file
df = pd.read_csv('output.txt')
df.head()

In [ ]:
df_filtered = df[df['BlockSize'] == 1]
plot_polynomial_fit(df_filtered, 'MatrixSize', 'Time(us)', degree=3)

In [ ]:
# Create a speedup dataframe for each block size
df.sort_values(by=['MatrixSize', 'BlockSize'], inplace=True)
df_speedup = {"MatrixSize": []}
for _, row in df.iterrows():
    matrix_size = row["MatrixSize"]
    if matrix_size not in df_speedup["MatrixSize"]:
        df_speedup["MatrixSize"].append(matrix_size)
    block_size = str(row["BlockSize"])
    if block_size not in df_speedup:
        df_speedup[block_size] = []
    df_speedup[block_size].append(row["Time(us)"])
df_speedup = pd.DataFrame(df_speedup)
df_speedup.head()

for block_size in df_speedup.columns[2:]:
    df_speedup[block_size] = df_speedup['1'] / df_speedup[block_size]

df_speedup


In [ ]:
# lineplot for all columns except MatrixSize    
for block_size in df_speedup.columns[2:]:
    sns.lineplot(data=df_speedup, x='MatrixSize', y=block_size, label=block_size, marker='o')
plt.ylabel('Speedup')
plt.legend()
plt.show()